In [207]:
import sqlite3
import psycopg2
import pandas as pd
import numpy as np
import math
from DataBase_functions import *
from DataBase_functions_testing import *

#Google sheed API
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

<p style="font-size: 30px; text-align: center; color: #20a7e5;">Creating data-frame</p>

<p style="font-size: 25px; color: #208ee5">Registro actividad todos</p>

In [208]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c" #Registro actividad todos: 12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #Dataframe con los datos de todos los meses
    df = pd.DataFrame(columns=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]) 
    meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
    for sheet in sheets:
        sheet_name = sheet.title 
        if sheet_name in meses:
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
            df = pd.concat([df, df_aux], ignore_index=True)
        else: 
            print(sheet_name, "is not a month")
            pass
    #df = df.drop_duplicates(subset='Código', keep='first')
    print(df.shape, "\n", df.columns)

    #Dataframe with 2024 sheet data
    sheet_2024 = spreadsheet.worksheet("2024")
    df_2024 = pd.DataFrame(sheet_2024.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
    #df_2024 = df_2024.drop_duplicates(subset='Código', keep='first')
    print(df_2024.shape, "\n", df_2024.columns)

    #Concatenate both dataframes
    df = pd.concat([df, df_2024], ignore_index=True)
    df = df.drop_duplicates(subset="Código", keep="first")
    print(df.shape)

    #Dataframe validation
    df.rename(columns={"Validada": "validada", "Responsable": "responsable", "Gestor DDBB": "DDBB", "Geografía": "geography", "Petición": "petition_code", "Feature": "feature", "SDATOOL": "sdatool", "Código": "DQDP_code", "Gestor BBDD": "DDBB", "Horas": "duration_time", "Comentarios": "description", "Fecha de alta": "fecha_in", "Fecha de fin": "fecha_out"}, inplace=True)

    df["fecha_in"] = df["fecha_in"].astype(str)
    df["Fecha incurrida"] = df["Fecha incurrida"].astype(str)
    df["fecha_out"] = df["fecha_out"].astype(str)
    df["UUAA"] = df["UUAA"].astype(str)
    df["DQDP_code"] = df["DQDP_code"].astype(str)
    df["sdatool"] = df["sdatool"].astype(str)
    df["feature"] = df["feature"].astype(str)
    df["geography"] = df["geography"].astype(str)
    df["DDBB"] = df["DDBB"].astype(str)
    df["Ámbito"] = df["Ámbito"].astype(str)
    df["responsable"] = df["responsable"].astype(str)
    df["validada"] = df["validada"].astype(str)
    df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
    df["description"] = df["description"].astype(str)
    df["petition_code"] = df["petition_code"].astype(str)

    df["version_date"] = "Nan"
    df["version"] = "Nan"
    df["petition_arq"] = "Nan"
    df["dev_master"] = "Nan"
    df["estado"] = "Nan"

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['2024 totales - Peticiones', '2024 totales - Peticiones - Pro', '2024', 'Gráficos Semanales - Procesos', '2024 - Procesos', 'Abril', 'Mayo', 'Julio', 'Junio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Responsables']
2024 totales - Peticiones is not a month
2024 totales - Peticiones - Pro is not a month
2024 is not a month
Gráficos Semanales - Procesos is not a month
2024 - Procesos is not a month
Responsables is not a month
(4371, 18) 
 Index(['Fecha de alta', 'Fecha incurrida', 'Fecha de fin', 'UUAA', 'Código',
       'SDATOOL', 'Feature', 'Petición', 'Geografía', 'Gestor BBDD', 'Ámbito',
       'Responsable', 'Validada', 'Horas', 'Comentarios', 'Estado', 'Días',
       ''],
      dtype='object')
(1974, 18) 
 Index(['Fecha de alta', 'Fecha incurrida', 'Fecha de fin', 'UUAA', 'Código',
       'SDATOOL', 'Feature', 'Petición', 'Geografía', 'Gestor BBDD', 'Ámbito',
       'Responsable', 'Validada', 'Horas', 'Comentarios', 'Estado', 'Días',
       ''

<p style="font-size: 25px; color: #208ee5; text-align: center;">Peticiones globales Oracle, Elastic Search y Mongo DB</p>

<p style="font-size: 25px; color: #208cc5;;">Oracle Physics</p>

In [209]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE" #Peticiones globales Oracle: "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_OG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_OG = pd.concat([df_OG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_OG["DDBB"] = "Oracle Physics"
    df_OG["geography"] = "Global"

    print(df_OG.shape)

    df_OG["UUAA"] = df_OG["UUAA"].astype(str)
    df_OG["Petición Jira"] = df_OG["Petición Jira"].astype(str)
    df_OG["Versión Repo"] = df_OG["Versión Repo"].astype(str)
    df_OG["Petición ARQ"] = df_OG["Petición ARQ"].astype(str)
    df_OG["Fecha cierre"] = df_OG["Fecha cierre"].astype(str)
    df_OG["Comentarios"] = df_OG["Comentarios"].astype(str)
    df_OG["Path"] = df_OG["Path"].astype(str)
    df_OG["DDBB"] = df_OG["DDBB"].astype(str)
    df_OG["geography"] = df_OG["geography"].astype(str)

    df_OG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['APIC', 'ATAU', 'B1DS', 'B43T', 'BJP9', 'CCUS', 'DAMS', 'DMTO', 'EGRP', 'EI4M', 'ESCD', 'FZHI', 'GDEL', 'GHDC', 'GJSX', 'GSIN', 'GUAR', 'H9IY', 'HFPP', 'HLCY', 'HPGR', 'IMPT', 'J6G7', 'K5VI', 'KAPI', 'KARC', 'KBGE', 'KBTQ', 'KCCX', 'KCDO', 'KCMC', 'KCMS', 'KCNC', 'KCNS', 'KCOG', 'KCOL', 'KCSN', 'KCSP', 'KDCL', 'KDFH', 'KDMG', 'KERF', 'KESJ', 'KFGC', 'KFRE', 'KFUL', 'KGCC', 'KGCI', 'KGTP', 'KIFV', 'KIVS', 'KLIM', 'KLNE', 'KMET', 'KMIC', 'KMOL', 'KMRK', 'KMYC', 'KNWD', 'KOMN', 'KONB', 'KPAG', 'KPAD', 'KPAY', 'KPDA', 'KPDR', 'KPFE', 'KPFM', 'KPIG', 'KPPS', 'KPST', 'KRBA', 'KRCE', 'KSAG', 'KSAN', 'KSDD', 'KSPH', 'KSRO', 'KUSU', 'KVCS', 'KXSG', 'KYSZ', 'KYOP', 'KYUB', 'MCRR', 'MFCG ', 'MK0W', 'MDDM', 'NI0C', 'O43T', 'PGVC', 'PIVU', 'PMSC', 'PMSV', 'P6ES', 'QWJR', 'QWPJ', 'RLR7', 'SWP3', 'S47B', 'UGX2', 'V34F', 'VAMO', 'W1BD', 'XBOB', 'ZGR7', 'XMBR', 'XR6Z', 'ZDNM', 'ZNXD']
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Re

<p style="font-size: 25px; color: #208cc5;;">Elastic Search</p>

In [210]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE" #Peticiones globales Mongo: "1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_EG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_EG = pd.concat([df_EG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_EG["DDBB"] = "Oracle Physics"
    df_EG["geography"] = "Global"

    #df = df_OG.drop_duplicates(subset='Código', keep='first')
    print(df_EG.shape)

    df_EG["UUAA"] = df_EG["UUAA"].astype(str)
    df_EG["Petición Jira"] = df_EG["Petición Jira"].astype(str)
    df_EG["Versión Repo"] = df_EG["Versión Repo"].astype(str)
    df_EG["Petición ARQ"] = df_EG["Petición ARQ"].astype(str)
    df_EG["Fecha cierre"] = df_EG["Fecha cierre"].astype(str)
    df_EG["Comentarios"] = df_EG["Comentarios"].astype(str)
    df_EG["Path"] = df_EG["Path"].astype(str)
    df_EG["DDBB"] = df_EG["DDBB"].astype(str)
    df_EG["geography"] = df_EG["geography"].astype(str)

    df_EG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['ATAU', 'F30V', 'EI4M', 'GHDC', 'GUAR', 'H9IY', 'IMPT', 'J6G7', 'JBSY', 'KAGE', 'KAPI', 'KBTQ', 'KCNS', 'KHR1', 'KIFV', 'KLNE', 'KMOL', 'KREC', 'KTIN', 'KTRA', 'KTR2', 'KXSG', 'KYGU', 'KYOP', 'L1WI', 'NI0C', 'O43T', 'OPEI', 'PGVC', 'S47B', 'XBMR', 'XPI2', 'YM3X', 'ZNXD']
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
(248, 10)


<p style="font-size: 25px; color: #208cc5;">Mongo DB</p>

In [211]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs" #Peticiones globales Mongo: "1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_MG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_MG = pd.concat([df_MG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_MG["DDBB"] = "Oracle Physics"
    df_MG["geography"] = "Global"
    
    print(df_MG.shape)

    df_MG["UUAA"] = df_MG["UUAA"].astype(str)
    df_MG["Petición Jira"] = df_MG["Petición Jira"].astype(str)
    df_MG["Versión Repo"] = df_MG["Versión Repo"].astype(str)
    df_MG["Petición ARQ"] = df_MG["Petición ARQ"].astype(str)
    df_MG["Fecha cierre"] = df_MG["Fecha cierre"].astype(str)
    df_MG["Comentarios"] = df_MG["Comentarios"].astype(str)
    df_MG["Path"] = df_MG["Path"].astype(str)
    df_MG["DDBB"] = df_MG["DDBB"].astype(str)
    df_MG["geography"] = df_MG["geography"].astype(str)

    df_MG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)
    
except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['KSKR', 'KPDR', 'F30V', 'KAPI', 'H9IY', 'KPAD', 'CHM8', 'XMBR', 'KCOG', 'KONL', 'KNWD', 'KIFV', 'KIFD', 'KPFM', 'KHR1', 'KARC', 'KATT']
(77, 9)


<p style="font-size: 25px; color: #20cff5;">All in one Data Frame</p>

In [212]:
df_g = pd.DataFrame(columns=list(df.columns))
df_g = pd.concat([df_g, df_OG, df_EG, df_MG], ignore_index=True)

df["fecha_in"] = df["fecha_in"].astype(str)
df["Fecha incurrida"] = df["Fecha incurrida"].astype(str)
df["fecha_out"] = df["fecha_out"].astype(str)
df["UUAA"] = df["UUAA"].astype(str)
df["DQDP_code"] = df["DQDP_code"].astype(str)
df["sdatool"] = df["sdatool"].astype(str)
df["feature"] = df["feature"].astype(str)
df["geography"] = df["geography"].astype(str)
df["DDBB"] = df["DDBB"].astype(str)
df["Ámbito"] = df["Ámbito"].astype(str)
df["responsable"] = df["responsable"].astype(str)
df["validada"] = df["validada"].astype(str)
df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
df["description"] = df["description"].astype(str)
df["version_date"] = df["version_date"].astype(str)
df["version"] = df["version"].astype(str)
df["petition_arq"] = df["petition_arq"].astype(str)
df["estado"] = df["estado"].astype(str)
df["petition_code"] = df["petition_code"].astype(str)
df["dev_master"] = "None"

df_g["fecha_in"] = df_g["fecha_in"].astype(str)
df_g["Fecha incurrida"] = df_g["Fecha incurrida"].astype(str)
df_g["fecha_out"] = df_g["fecha_out"].astype(str)
df_g["UUAA"] = df_g["UUAA"].astype(str)
df_g["DQDP_code"] = df_g["DQDP_code"].astype(str)
df_g["sdatool"] = df_g["sdatool"].astype(str)
df_g["feature"] = df_g["feature"].astype(str)
df_g["geography"] = df_g["geography"].astype(str)
df_g["DDBB"] = df_g["DDBB"].astype(str)
df_g["Ámbito"] = df_g["Ámbito"].astype(str)
df_g["responsable"] = df_g["responsable"].astype(str)
df_g["validada"] = df_g["validada"].astype(str)
df_g["duration_time"] = pd.to_numeric(df_g["duration_time"], errors="coerce")
df_g["description"] = df_g["description"].astype(str)
df_g["version_date"] = df_g["version_date"].astype(str)
df_g["version"] = df_g["version"].astype(str)
df_g["petition_arq"] = df_g["petition_arq"].astype(str)
df_g["estado"] = df_g["estado"].astype(str)
df_g["petition_code"] = df_g["petition_code"].astype(str)
df_g["dev_master"] = "None"

####################################################################################################
dataframes_using = [df, df_g]
for i in dataframes_using:
    for row in i.iterrows(): 
        for key, value in dict(row[1]).items():
            if value == "Nan" or value == None or value == "" or value == "nan" or value == "None" or value == "none" or value == "NONE" or value == np.nan: 
                if key == "fecha_in" or key == "fecha_out" or key == "Fecha incurrida" or key == "UUAA" or key == "DQDP_code" or key == "sdatool" or key == "feature" or key == "geography" or key == "DDBB" or key == "Ámbito" or key == "responsable" or key == "validada" or key == "description" or key == "version_date" or key == "version" or key == "petition_arq" or key == "Estado" or key == "petition_code" or key == "dev_master":
                    i.at[row[0], key] = "None"
                if key == "duration_time":
                    i.at[row[0], key] = None
                else: 
                    pass

df_g.drop_duplicates(subset=["petition_code"], keep="first", inplace=True)
df_g = df_g[df_g["petition_code"] != "None"]


#df = pd.concat([df, df_g], ignore_index=True)



In [213]:
df_none_count = {i: 0 for i in list(df_g.columns)}
for row in df_g.iterrows(): 
    for key, value in dict(row[1]).items():
        if value == "None" or value == None:
            df_none_count[key] += 1
        else: 
            pass
print(df_g.shape, df.shape)
df_none_count

(1152, 24) (1739, 23)


{'fecha_in': 1152,
 'Fecha incurrida': 1152,
 'fecha_out': 40,
 'UUAA': 1,
 'DQDP_code': 1152,
 'sdatool': 1152,
 'feature': 1152,
 'petition_code': 0,
 'geography': 0,
 'DDBB': 0,
 'Ámbito': 1152,
 'responsable': 1152,
 'validada': 1152,
 'duration_time': 0,
 'description': 1049,
 'Estado': 0,
 'Días': 0,
 '': 0,
 'version_date': 1152,
 'version': 3,
 'petition_arq': 48,
 'dev_master': 1152,
 'estado': 0,
 'Path': 0}

In [218]:
df_definitive = pd.DataFrame(columns=list(df.columns))
petitions = set(df_g["petition_code"]).union(set(df["petition_code"]))
petitions

{'OTRSSD-10903',
 'OTRSSD-13178',
 'OTRSSD-15632',
 'OTRSSD-13078',
 'REQ000101626280',
 'OTRSSD-5197',
 'OTRSSD-16053',
 'OTRSSD-8582',
 'OTRSSD-17848',
 'OTRSSD-17361',
 'OTRSSD-13262',
 'OTRSSD-5588',
 'OTRSSD-8834',
 'SGS-16039',
 'OTRSSD-16473',
 'OTRSSD-15595',
 'OTRSSD-13123',
 'OTRSSD-7996',
 'OTRSSD-10461',
 'SGS-14883',
 'WO0000101356732',
 'REQ000101635252',
 'OTRSSD-15986',
 'OTRSSD-10312',
 'SGS-17978',
 'OTRSSD-15232',
 'OTRSSD-17548',
 'OTRSSD-13321',
 'OTRSSD-11568',
 'OTRSSD-15566',
 'OTRSSD-15564',
 'OTRSSD-12429',
 'OTRSSD-3965',
 'OTRSSD-15782',
 'OTRSSD-16476',
 'TCTF_20241209144500',
 'WO0000101319132',
 'REQ000101109752',
 'OTRSSD-4801',
 'OTRSSD-16550',
 'OTRSSD-17992',
 'INCL_20241210111237',
 'SGS-9547',
 'OTRSSD-16345',
 'OTRSSD-13989',
 'OTRSSD-17055',
 'OTRSSD-13274',
 'OTRSSD-14966',
 'OTRSSD-11673',
 'SGS-17062',
 'OTRSSD-14205',
 'OTRSSD-14917',
 'ADEX_20241001115523',
 'OTRSSD-15822',
 'REQ000101117160',
 'OTRSSD-17172',
 'REQ000101446933',
 'OTRSSD-174

In [215]:
print(df_g.columns, df.columns)

Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'duration_time', 'description', 'Estado',
       'Días', '', 'version_date', 'version', 'petition_arq', 'dev_master',
       'estado', 'Path'],
      dtype='object') Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'duration_time', 'description', 'Estado',
       'Días', '', 'version_date', 'version', 'petition_arq', 'dev_master',
       'estado'],
      dtype='object')


---
---

In [216]:
'''
for row in df.iterrows():
    insert_data_testing(dict(row[1]))
'''

'\nfor row in df.iterrows():\n    insert_data_testing(dict(row[1]))\n'

<p style="font-size: 40px; color: grey;">Testing</p>

In [217]:
diccionario = {"Nombre": "Matias", "Apellido": "Blaquier", "Edad": 27}
for key, value in diccionario.items(): 
    print(value)

###########
str_test = " "
print(len(str_test))
print(len(str_test.strip()))

Matias
Blaquier
27
1
0
